# Delaunay-Watershed for segmentation masks compression

In this notebook:
- We present how to compress segmentation masks into a multimaterial mesh. Here we apply it on nuclei segmentation.
- We show that we can recompute the mask images from the mesh.

In this example, the compressed data is 128 times smaller than the original data.

# 0) Install Delaunay-Watershed

Along with tools for this notebook specifically.

In [ ]:
pip install delaunay-watershed-3d csbdeep napari[all]

## 1) Download nuclei data from Stardist 

In [ ]:
from glob import glob

import numpy as np
from csbdeep.utils import Path, download_and_extract_zip_file

download_and_extract_zip_file(
    url       = "https://github.com/stardist/stardist/releases/download/0.3.0/demo3D.zip",
    targetdir = "../data",
    verbose   = 1)


X = sorted(glob("../data/train/images/*.tif"))
Y = sorted(glob("../data/train/masks/*.tif"))
assert all(Path(x).name==Path(y).name for x,y in zip(X,Y, strict=False))

## 2) Compute multimaterial mesh with Delaunay Watershed

A mesh is created via a "Mesh Reconstruction Algorithm". Although many variations can be created, we provide a default Mesh Reconstruction Algorithm:

In [ ]:
from dw3d import get_default_mesh_reconstruction_algorithm

dist = 3
mesh_reconstruction_algorithm = get_default_mesh_reconstruction_algorithm(min_distance=dist, print_info=True)

But instead of a mesh, you can choose to obtain a compressed version of the segmentation masks:

In [ ]:
import skimage.io as io
from scipy.ndimage import zoom

dist = 2
i = 2
initial_segmented_image = zoom(io.imread(Y[i]), [2,1,1],order = 0)


compressed_segmentation_dict = mesh_reconstruction_algorithm.compress_segmentation_mask(initial_segmented_image)

In [ ]:
print(compressed_segmentation_dict)

# Note that the last compressed segmentation dict can also be obtained via
compressed_segmentation_dict = mesh_reconstruction_algorithm.last_compressed_segmentation

In [ ]:
# It can also be saved on disk:
from dw3d import save_compressed_segmentation

save_compressed_segmentation("dict_mask.npy", compressed_segmentation_dict)

## 3) Reconstruct segmentation masks from the mesh. Visualize in napari for comparison

In [ ]:
from dw3d import reconstruct_mask_from_dict, reconstruct_mask_from_saved_file_dict

segmented_image_reconstructed = reconstruct_mask_from_saved_file_dict("dict_mask.npy")
# or if you still have the dict directly
# segmented_image_reconstructed = reconstruct_mask_from_dict(compressed_segmentation_dict)

In [ ]:
import napari

v = napari.view_image(segmented_image_reconstructed)
v.add_image(initial_segmented_image)

## 4) Compare files size: mesh VS masks

In [ ]:

npdict = np.load("dict_mask.npy",allow_pickle=True).item()
dict_memory_footprint = npdict["points"].nbytes + npdict["triangles"].nbytes + npdict["seeds"].nbytes + np.array(npdict["image_shape"]).nbytes
print("Size of Delaunay-Watershed data:",dict_memory_footprint,"bytes")
print("Size of the original segmentation image:",segmented_image_reconstructed.nbytes,"bytes")
print("Relative size:", dict_memory_footprint/segmented_image_reconstructed.nbytes)
print("Compression factor is:", segmented_image_reconstructed.nbytes/dict_memory_footprint)